In [1]:
# Cài đặt thư viện bổ sung nếu chưa có
%pip install seqeval transformers datasets torch "accelerate>=0.26.0"

import json
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import LayoutLMForTokenClassification, LayoutLMTokenizerFast
from transformers import Trainer, TrainingArguments
from seqeval.metrics import classification_report
import numpy as np

# Đường dẫn (giống bài trước)
DATA_ROOT = "../data"
PROCESSED_DIR = os.path.join(DATA_ROOT, "processed")
OUTPUT_DIR = "../models/layoutlm_best"

You should consider upgrading via the 'c:\Users\admin\Documents\HUMG_SUBJECT\Data_Mining\Project\invoice-extraction-project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


c:\Users\admin\Documents\HUMG_SUBJECT\Data_Mining\Project\invoice-extraction-project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load label map
with open(os.path.join(PROCESSED_DIR, "label_map.json"), 'r', encoding='utf-8') as f:
    label2id = json.load(f)

# Chuyển đổi key từ string sang int (vì JSON lưu key là string)
# id2label cần thiết để mô hình hiểu output số 1 là "SELLER"
id2label = {int(v): k for k, v in label2id.items()}

print("Các nhãn:", list(label2id.keys()))

# Load train và validation data
with open(os.path.join(PROCESSED_DIR, "train.json"), 'r', encoding='utf-8') as f:
    train_data = json.load(f)

with open(os.path.join(PROCESSED_DIR, "validation.json"), 'r', encoding='utf-8') as f:
    val_data = json.load(f)

print(f"Số lượng Train: {len(train_data)}")
print(f"Số lượng Val: {len(val_data)}")

Các nhãn: ['OTHER', 'SELLER', 'ADDRESS', 'TIMESTAMP', 'TOTAL_COST']
Số lượng Train: 920
Số lượng Val: 231


In [3]:
# Sử dụng Tokenizer của LayoutLM
tokenizer = LayoutLMTokenizerFast.from_pretrained("microsoft/layoutlm-base-uncased")

class InvoiceDataset(Dataset):
    def __init__(self, data, tokenizer, label2id, max_len=512):
        self.data = data
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        words = item['tokens']
        boxes = item['bboxes']
        # Chuyển label id gốc sang list
        original_labels = item['ner_tags'] 

        # Tokenize (quan trọng: phải set is_split_into_words=True)
        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            return_offsets_mapping=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )

        # Chúng ta cần map các box và label gốc sang các sub-tokens
        word_ids = encoding.word_ids() # Ví dụ: [None, 0, 1, 1, 2, None...]
        
        token_boxes = []
        token_labels = []
        
        # Box mặc định cho các token đặc biệt (CLS, SEP, PAD)
        pad_box = [0, 0, 0, 0]

        for word_idx in word_ids:
            if word_idx is None:
                # Token đặc biệt (CLS, SEP, PAD)
                token_boxes.append(pad_box)
                token_labels.append(-100) # -100 để PyTorch bỏ qua khi tính loss
            else:
                # Token thuộc về một từ gốc
                # Lấy box và label của từ gốc gán cho token này
                token_boxes.append(boxes[word_idx])
                token_labels.append(original_labels[word_idx])

        # Chuyển về dạng Tensor
        item_dict = {key: val.squeeze() for key, val in encoding.items()}
        item_dict['bbox'] = torch.tensor(token_boxes)
        item_dict['labels'] = torch.tensor(token_labels)
        
        # Xóa offset_mapping không cần thiết cho model
        if 'offset_mapping' in item_dict:
            del item_dict['offset_mapping']

        return item_dict

# Tạo dataset object
train_dataset = InvoiceDataset(train_data, tokenizer, label2id)
val_dataset = InvoiceDataset(val_data, tokenizer, label2id)

# Kiểm tra thử 1 mẫu
sample = train_dataset[0]
print("Kích thước Input IDs:", sample['input_ids'].shape)
print("Kích thước Bbox:", sample['bbox'].shape)
print("Kích thước Labels:", sample['labels'].shape)

Kích thước Input IDs: torch.Size([512])
Kích thước Bbox: torch.Size([512, 4])
Kích thước Labels: torch.Size([512])


In [4]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Loại bỏ các label -100 (token đặc biệt) để tính điểm cho chính xác
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }
    return results

In [5]:
# Ensure accelerate is installed for Trainer


# Load Model Pretrained
model = LayoutLMForTokenClassification.from_pretrained(
    "microsoft/layoutlm-base-uncased",
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label
)

# Cấu hình tham số training
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    max_steps=1000, # Chạy demo 1000 bước (hoặc đổi thành num_train_epochs=5)
    per_device_train_batch_size=4, # Nếu GPU yếu (với 4GB VRAM) thì giảm xuống 2
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    eval_strategy="steps", # Đánh giá sau mỗi X bước
    eval_steps=100,
    save_steps=100,
    logging_steps=50,
    load_best_model_at_end=True, # Tự động load model tốt nhất khi xong
    metric_for_best_model="f1",
    save_total_limit=2, # Chỉ lưu 2 checkpoint gần nhất để tiết kiệm ổ cứng
    remove_unused_columns=False, # Cần thiết cho LayoutLM
    push_to_hub=False,
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Bắt đầu training
print("🚀 Bắt đầu training...")
trainer.train()

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\admin\AppData\Local\Temp\ipykernel_26496\3761270475.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\admin\Documents\HUMG_SUBJECT\Data_Mining\Project\invoice-extraction-project\venv\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


🚀 Bắt đầu training...


Step,Training Loss,Validation Loss,Precision,Recall,F1
100,0.266800,0.191344,0.794275,0.683077,0.734491
200,0.160200,0.133586,0.787250,0.816923,0.801812
300,0.127100,0.106869,0.823746,0.859231,0.841114
400,0.083900,0.105092,0.876448,0.873077,0.874759
500,0.108700,0.102473,0.868914,0.892308,0.880455
600,0.058200,0.095140,0.872837,0.892308,0.882465
700,0.050700,0.094562,0.869955,0.895385,0.882487
800,0.040600,0.091570,0.880000,0.896923,0.888381
900,0.041400,0.091501,0.876589,0.901538,0.888889
1000,0.035900,0.097258,0.880451,0.900769,0.890494


c:\Users\admin\Documents\HUMG_SUBJECT\Data_Mining\Project\invoice-extraction-project\venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: OTHER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\admin\Documents\HUMG_SUBJECT\Data_Mining\Project\invoice-extraction-project\venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: TIMESTAMP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\admin\Documents\HUMG_SUBJECT\Data_Mining\Project\invoice-extraction-project\venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: TOTAL_COST seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\admin\Documents\HUMG_SUBJECT\Data_Mining\Project\invoice-extraction-project\venv\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADDRESS seems not to be NE tag.
  warnings.warn('{} seems not to be N

TrainOutput(global_step=1000, training_loss=0.12633006513118744, metrics={'train_runtime': 9498.2742, 'train_samples_per_second': 0.421, 'train_steps_per_second': 0.105, 'total_flos': 1052472569856000.0, 'train_loss': 0.12633006513118744, 'epoch': 4.3478260869565215})

In [6]:
# Lưu model và tokenizer vào thư mục output
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"✅ Đã lưu model thành công tại: {OUTPUT_DIR}")

✅ Đã lưu model thành công tại: ../models/layoutlm_best
